In [1]:
from vispy import app, scene, visuals,use
from vispy.util.filter import gaussian_filter
import numpy as np
import vispy.io as io
from vispy.scene.visuals import Ellipse
# if offscreen 
# use egl or osmesa
import filepath
import os
from util.repo import res_root
from util.texture import np2texture, rgba_mpl2pd3d, texture_load_np
from panda3d.core import PNMImage, Texture
from panda3d_game.app import *
from demos.ball_room import MassedBall,tmoon
from panda3d.core import (
    Geom,
    GeomNode,
    GeomTriangles,
    GeomVertexData,
    GeomVertexFormat,
    GeomVertexWriter,
    GeomEnums
)
from panda3d.core import (
    NodePath,
    CardMaker,
    Texture,
    CardMaker,
    Point2,
    Vec3
)
from vispy.visuals.transforms.linear import MatrixTransform
from art.basic import create_cylinder_node, create_sphere_node
# import direct.directbase.DirectStart
from util.physics import autocomplete_units, G_val, getG
from panda3d.core import AmbientLight
from sympy.physics.units import (
    kilometer, meter,centimeter,
    gram, kilogram, tonne,
    newton, second
)
from util.rendering import *


from vispyutil.scene import UnboundedTurnableCam
from vispyutil.canvas import SynchronizedCanvas
# from vispyutil.showbase import CanvasBackgroundShowBase
from panda3d_game.app import CanvasBackgroundShowBase, QControl

class StarCanvas(SynchronizedCanvas):
    def __init__(self, n_stars, rho=10):
        SynchronizedCanvas.__init__(self)
        self.n_stars = n_stars 
        # randomly pick stars
        np.random.seed(0)  # todo: use torch random state
        cos_pos_theta = np.random.uniform(-1,1, n_stars)
        pos_theta = np.arccos(cos_pos_theta)
        pos_phi = np.random.uniform(0,2*np.pi, n_stars)
        sizes = np.random.uniform(0, 5, n_stars) 
        r = rho * np.sin(pos_theta)
        x = r * np.cos(pos_phi)
        y = r * np.sin(pos_phi)
        z = rho * np.cos(pos_theta)
        positions = np.vstack([x,y,z]).T
        scatter = scene.visuals.Markers()
        scatter.set_data(positions, edge_color=None, face_color='white', size=sizes, symbol='o')
        # Add scatter plot to the view
        self.view.add(scatter)
        # TODO: milky way


class StarSphere(ControlShowBase, CanvasBackgroundShowBase):
    def __init__(self):
        ControlShowBase.__init__(self)
        self.stars_canvas = StarCanvas(60000)
        CanvasBackgroundShowBase.__init__(self,self.stars_canvas)

class PlanetSphere(StarSphere,PhysicsShowBase):
    def __init__(self):
        StarSphere.__init__(self)
        PhysicsShowBase.__init__(self)
        self.bullet_world.setGravity((0,0,0))
        self.unit = {
            "mass" : tonne,
            "length" : 100*meter,
            "time": 1 * second,
            # "force" : sp.Number(1e3) * newton
        }
        autocomplete_units(self.unit)
        self.planet1 = MassedBall(
            name="planet1",
            radius=1000*meter,
            mass=1e6*tonne,
            units=self.unit
        )
        self.planet1.reparentTo(self.render)
        self.planet1.set_texture(tmoon)
        self.planet1.setScale(10)
        self.planet1.setPos(0,10,0)

if __name__ == "__main__":
    from ui.abstract_ui import InterfacePlaceHolder
    import builtins
    import traceback
    # interface = InterfacePlaceHolder()
    try:
        with PlanetSphere() as app_:
            # console = PhyscRoomConsole(showbase=app)
            # interface = CMDInterface(console=console)
            # .start()
            # start a thread of app
            app_.run()
    except Exception as e:
        print(e)
        print(traceback.format_exc())
    finally:
        if hasattr(builtins, 'base'):
            builtins.base.destroy()
        # interface.join()


Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager


init ContextShowBase
---set ref---:render,<class 'panda3d.core.NodePath'>
---<__main__.PlanetSphere object at 0x7f7addf50c50> run(), at 2024-10-06 05:27:05.276180---


/media/ywatcher/ExtDisk1/LDisk/Packages/game_env/env/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1711322746054/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


---<__main__.PlanetSphere object at 0x7f7addf50c50> destroy at 2024-10-06 05:27:08.864654---
---<__main__.PlanetSphere object at 0x7f7addf50c50> destroyed at 2024-10-06 05:27:08.865072, exit---


SystemExit: 

/media/ywatcher/ExtDisk1/LDisk/Packages/game_env/env/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
use( 'glfw')

In [65]:
cam = UnboundedTurnableCam()

aclled


In [66]:
cam.__dict__

{'_name': None,
 '_visible': True,
 '_canvas': None,
 '_document_node': None,
 '_scene_node': None,
 '_opacity': 1.0,
 '_order': 0,
 '_picking': False,
 '_clippers': <WeakKeyDictionary at 0x7f7de6f3fd90>,
 '_clip_children': False,
 '_clipper': None,
 'transforms': <vispy.visuals.transforms.transform_system.TransformSystem at 0x7f7de6f3eb50>,
 'events': <vispy.util.event.EmitterGroup at 0x7f7de6f65490>,
 '_children': [],
 '_transform': MatrixTransform(matrix=[[1.0, 0.0, 0.0, 0.0],
                         [0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0.0],
                         [0.0, 0.0, 0.0, 1.0]] at 0x7f7e25d7c110),
 '_parent': None,
 '_document': None,
 '_viewbox': None,
 '_linked_cameras': {},
 '_linked_cameras_no_update': None,
 '_pre_transform': None,
 '_viewbox_tr': <STTransform scale=[1. 1. 1. 1.] translate=[0. 0. 0. 0.] at 0x140179277833232>,
 '_projection': MatrixTransform(matrix=[[1.0, 0.0, 0.0, 0.0],
                         [0.0, 1.0, 0.0, 0.0],
       

In [67]:
UnboundedTurnableCam._state_props

('scale_factor', 'center', 'fov', 'elevation', 'azimuth', 'roll')

In [68]:
cam._elevation

30.0

In [57]:
type(canvas.view.camera.transformii)

NameError: name 'canvas' is not defined

In [25]:
vispy.visuals.transforms.linear.MatrixTransform(np.array([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,0],
    [0,0,0,1]
]))

MatrixTransform(matrix=[[1, 0, 0, 0],
                        [0, 1, 0, 0],
                        [0, 0, 1, 0],
                        [0, 0, 0, 1]] at 0x7f3442ce7850)

In [5]:
import vispy
import panda3d